In [1]:
import h5py
# import openslide
import torch
import pandas as pd
from prototype_visualization_utils import get_panther_encoder, visualize_categorical_heatmap, get_mixture_plot, get_default_cmap

import sys
sys.path.append('../')
from mil_models.tokenizer import PrototypeTokenizer

In [2]:
### Loading PANTHER Encoder
proto_path = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_k=0\prototypes\prototypes_c16_uni_kmeans_num_1.0e+06.pkl' # 5x uni
# proto_path = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_10x_k=0\prototypes\prototypes_c16_uniextracted_mag10x_patch224_fp_kmeans_num_1.0e+06.pkl' # 10x uni
# proto_path = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_2.5x_k=0\prototypes\prototypes_c16_uniextracted_mag2.5x_patch224_fp_kmeans_num_1.0e+06.pkl' # 2.5x uni
# , model_config='PANTHER_fa_pt' out_type='allcat', 
panther_encoder = get_panther_encoder(in_dim=1024, p=16, proto_path=proto_path, config_dir=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs', model_config='PANTHER_fa_pt', out_type='allcat')

📌 Loaded config path: C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs\PANTHER_fa_pt\config.json
📌 Loaded config_dict from JSON: {'in_dim': 1024, 'n_classes': 2, 'heads': 1, 'em_iter': 1, 'tau': 0.001, 'ot_eps': 0.1, 'n_fc_layers': 0, 'dropout': 0.25, 'out_type': 'allcat', 'out_size': 8, 'load_proto': False, 'proto_path': '.', 'fix_proto': False}
Loading prototypes from C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_k=0\prototypes\prototypes_c16_uni_kmeans_num_1.0e+06.pkl


creating proto maps + mixture plots

In [ ]:
import os
import h5py
import torch
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from prototype_visualization_utils import get_default_cmap, get_mixture_plot

# -----------------------
# Paths
# -----------------------
feats_h5_dir = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\uniextracted_mag5x_patch224_fp\feats_h5'
save_root = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\test_visualizations'
test_csv_path = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_k=0\test.csv'  # <-- your CSV path

os.makedirs(save_root, exist_ok=True)

# -----------------------
# Load test slide list
# -----------------------
df_test = pd.read_csv(test_csv_path)
test_slides = set(df_test['slide_id'].tolist())  # no normalization

print(f"Found {len(df_test)} test slides.")

# -----------------------
# Settings
# -----------------------
patch_size = 224
alpha = 1
scale = 0.25

# -----------------------
# Loop through features
# -----------------------
for fname in os.listdir(feats_h5_dir):
    if not fname.endswith('.h5'):
        continue

    slide_id = fname.replace('.h5', '')
    if slide_id not in test_slides:
        continue  # Skip if not in test set

    slide_dir = os.path.join(save_root, slide_id)
    if os.path.exists(slide_dir):
        print(f"⏭ Skipping {slide_id} (already exists)")
        continue

    os.makedirs(slide_dir, exist_ok=True)
    h5_path = os.path.join(feats_h5_dir, fname)
    print(f"Processing {slide_id}...")

    # -----------------------
    # Load features & coords
    # -----------------------
    with h5py.File(h5_path, 'r') as h5:
        coords = h5['coords'][:]
        feats = torch.Tensor(h5['features'][:])

    # -----------------------
    # Inference
    # -----------------------
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()
        out = info['repr']

        tokenizer = PrototypeTokenizer(out_type='allcat', p=qq.shape[1])
        mus, pis, sigmas = tokenizer.forward(out)
        mus = mus[0].detach().cpu().numpy()
        cluster_labels = qq.argmax(axis=1)

    # -----------------------
    # Prototype map
    # -----------------------
    x_min, y_min = coords.min(axis=0)
    coords -= np.array([x_min, y_min])
    max_x, max_y = coords.max(axis=0) + patch_size
    canvas_w, canvas_h = int(max_x), int(max_y)

    canvas = Image.new('RGBA', (canvas_w, canvas_h), (255, 255, 255, 255))
    draw = ImageDraw.Draw(canvas, 'RGBA')
    cmap = get_default_cmap(int(cluster_labels.max()) + 1)

    for (x, y), label in zip(coords, cluster_labels):
        color = cmap[label]
        rgba = color + (int(255 * alpha),)
        draw.rectangle([x, y, x + patch_size, y + patch_size], fill=rgba)

    if scale < 1.0:
        canvas = canvas.resize((int(canvas_w * scale), int(canvas_h * scale)))
    canvas.save(os.path.join(slide_dir, 'prototype_map.png'))

    # -----------------------
    # Mixture plot
    # -----------------------
    mixture_plot = get_mixture_plot(mus)
    mixture_plot.savefig(os.path.join(slide_dir, 'mixture_plot.png'), bbox_inches='tight')
    plt.close(mixture_plot)

print("✅ All test visualizations saved.")


Found 33 test slides.
Processing FA 56B...

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.2569
Processing FA 57B...

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 1.9535
Processing FA 58B...

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.4151
Processing FA 59B...

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.0455
Processing FA 60 B...

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.1538
Processing FA 61 B...

=== map_em called ===

=== mog_eval called =

debugging my proto map code that works on our dataset

In [ ]:
import os
import h5py
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from prototype_visualization_utils import get_default_cmap, get_mixture_plot

# Setup paths
feats_h5_dir = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide'
save_root = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\visualizations'
os.makedirs(save_root, exist_ok=True)

# Settings
patch_size = 224
alpha = 1
scale = 0.25

# Loop through all h5 feature files
for fname in os.listdir(feats_h5_dir):
    if not fname.endswith('.h5'):
        continue

    slide_id = fname.replace('.h5', '').replace(' ', '_')
    h5_path = os.path.join(feats_h5_dir, fname)
    slide_dir = os.path.join(save_root, slide_id)
    os.makedirs(slide_dir, exist_ok=True)

    print(f"Processing {slide_id}...")

    # Load features and coords
    with h5py.File(h5_path, 'r') as h5:
        coords = h5['coords'][:]
        feats = torch.Tensor(h5['features'][:])

    # Inference
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()
        print('qq.shape before tokenizer', qq.shape)  # debug
        print('qq before tokenizer', qq)  # debug
        out = info['repr']
        print('out.shape', out.shape)  # debug
        print('out before tokenizer', out)  # debug

        tokenizer = PrototypeTokenizer(out_type='allcat', p=qq.shape[1])
        mus, pis, sigmas = tokenizer.forward(out)
        print('mus.shape', mus.shape)  # debug
        print('pis.shape', pis.shape)  # debug
        print('sigmas.shape', sigmas.shape)  # debug
        print('mus before detach', mus)  # debug
        mus = mus[0].detach().cpu().numpy()
        print('mus after detach', mus)  # debug
        cluster_labels = qq.argmax(axis=1)
        print('cluster_labels.shape', cluster_labels.shape)  # debug
        print('cluster_labels before detach', cluster_labels)  # debug

for i in range(min(5, len(qq))):
    coord = coords[i]
    scores = qq[i]
    max_idx = scores.argmax()
    print(f"Patch {i} @ coord {tuple(coord)} → max proto: {max_idx}, score: {scores[max_idx]:.4f}, full: {np.round(scores, 3)}")


selecting top 4 patches

In [18]:
def visualize_top_k_patches_per_prototype_from_qq(
    h5_feats_fpath,
    patch_dir,
    panther_encoder,
    top_k=4,
    patch_size=224,
    save_path=None
):
    import glob
    import matplotlib.pyplot as plt
    import numpy as np
    import torch
    import h5py
    import os

    # Load patch-level features
    with h5py.File(h5_feats_fpath, 'r') as h5:
        coords = h5['coords'][:]
        feats = torch.Tensor(h5['features'][:])

    # Get prototype assignments
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()  # shape: (N, p)

    n_patches, n_protos = qq.shape

    # Select top-k patches per prototype (skip inactive prototypes)
    proto_to_indices = {}
    for proto in range(n_protos):
        proto_scores = qq[:, proto]
        if np.max(proto_scores) < 1e-5:  # Skip prototypes with no significant activation
            continue

        nonzero_indices = np.where(proto_scores > 1e-3)[0]
        if len(nonzero_indices) == 0:
            continue

        sorted_indices = nonzero_indices[np.argsort(-proto_scores[nonzero_indices])]
        proto_to_indices[proto] = sorted_indices[:top_k]

    active_protos = sorted(proto_to_indices.keys())
    if len(active_protos) == 0:
        print("⚠️ No active prototypes found in this slide.")
        return

    # Plotting
    fig, axs = plt.subplots(len(active_protos), top_k, figsize=(top_k * 3, len(active_protos) * 3))
    if len(active_protos) == 1:
        axs = np.expand_dims(axs, axis=0)

    for row_idx, proto_idx in enumerate(active_protos):
        indices = proto_to_indices[proto_idx]
        for col_idx in range(top_k):
            ax = axs[row_idx, col_idx]
            if col_idx >= len(indices):
                ax.axis('off')
                continue

            patch_idx = indices[col_idx]
            x, y = coords[patch_idx]

            pattern = f'*_x{int(x)}_y{int(y)}.npy'
            matches = glob.glob(os.path.join(patch_dir, pattern))
            if not matches:
                ax.axis('off')
                continue

            patch = np.load(matches[0])
            ax.imshow(patch.astype(np.uint8))
            if col_idx == 0:
                ax.set_title(f'Proto {proto_idx}')
            ax.axis('off')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"Saved top-k patches visualization to {save_path}")
    plt.close(fig)


creating 16x8 plot (top 4 + 4 random)

In [3]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import torch
import h5py
import random

def visualize_top_k_patches_per_prototype_from_qq(
    h5_feats_fpath,
    patch_dir,
    panther_encoder,
    top_k=4,
    patch_size=224,
    save_path=None
):
    # Load patch-level features
    with h5py.File(h5_feats_fpath, 'r') as h5:
        coords = h5['coords'][:]
        feats = torch.Tensor(h5['features'][:])

    # Get prototype assignments
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()  # shape: (N, p)

    n_patches, n_protos = qq.shape
    total_k = top_k * 2  # 4 top + 4 random

    # Select 4 top + 4 random per prototype
    proto_to_indices = {}
    for proto in range(n_protos):
        proto_scores = qq[:, proto]
        if np.max(proto_scores) < 1e-5:
            continue

        nonzero_indices = np.where(proto_scores > 1e-3)[0]
        if len(nonzero_indices) == 0:
            continue

        sorted_indices = nonzero_indices[np.argsort(-proto_scores[nonzero_indices])]
        top_indices = sorted_indices[:top_k]

        remaining = list(set(nonzero_indices) - set(top_indices))
        if len(remaining) >= top_k:
            random_indices = random.sample(remaining, top_k)
        else:
            random_indices = remaining

        combined_indices = list(top_indices) + list(random_indices)
        proto_to_indices[proto] = combined_indices

    active_protos = sorted(proto_to_indices.keys())
    if len(active_protos) == 0:
        print("⚠️ No active prototypes found in this slide.")
        return

    # Plotting
    fig, axs = plt.subplots(len(active_protos), total_k, figsize=(total_k * 2.5, len(active_protos) * 2.5))
    if len(active_protos) == 1:
        axs = np.expand_dims(axs, axis=0)

    for row_idx, proto_idx in enumerate(active_protos):
        indices = proto_to_indices[proto_idx]
        for col_idx in range(total_k):
            ax = axs[row_idx, col_idx]
            if col_idx >= len(indices):
                ax.axis('off')
                continue

            patch_idx = indices[col_idx]
            x, y = coords[patch_idx]
            pattern = f'*_x{int(x)}_y{int(y)}.npy'
            matches = glob.glob(os.path.join(patch_dir, pattern))

            if not matches:
                ax.axis('off')
                continue

            patch = np.load(matches[0])
            ax.imshow(patch.astype(np.uint8))
            if col_idx == 0:
                ax.set_title(f'Proto {proto_idx}')
            elif col_idx == top_k:
                ax.set_title('↓ Random')

            ax.axis('off')

    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"✅ Saved patch visualization to: {save_path}")
    plt.close()


8 x 16 plot

In [25]:
import os, glob, random, h5py, torch
import numpy as np
import matplotlib.pyplot as plt

def visualize_top_k_patches_per_prototype_from_qq(
    h5_feats_fpath,
    patch_dir,
    panther_encoder,
    top_k=4,
    patch_size=224,
    save_path=None
):
    # --- load features ---
    with h5py.File(h5_feats_fpath, 'r') as h5:
        coords = h5['coords'][:]
        feats = torch.tensor(h5['features'][:])

    # --- get assignments ---
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()  # (N_patches, N_protos)

    n_patches, n_protos = qq.shape
    total_k = top_k * 2  # top + random rows

    # --- select patches per proto (top_k + random_k) ---
    proto_to_indices = {}
    for proto in range(n_protos):
        ps = qq[:, proto]
        if ps.max() < 1e-5:
            continue
        nz = np.where(ps > 1e-3)[0]
        if len(nz) == 0:
            continue
        sorted_idx = nz[np.argsort(-ps[nz])]
        top_idx = list(sorted_idx[:top_k])
        remain = list(set(nz) - set(top_idx))
        rand_idx = random.sample(remain, min(top_k, len(remain)))
        proto_to_indices[proto] = top_idx + rand_idx

    # active_protos = sorted(proto_to_indices.keys())
    # if not active_protos:
    #     print("⚠️ No active prototypes found in this slide.")
    #     return

    all_protos = list(range(n_protos))  # fixed proto order
    fig, axs = plt.subplots(total_k, n_protos,
                            figsize=(n_protos * 2, total_k * 2))

    for col, proto in enumerate(all_protos):
        indices = proto_to_indices.get(proto, [])
        for row in range(total_k):
            ax = axs[row, col]
            if row >= len(indices):
                ax.axis('off')
                continue

    # # --- make grid: rows = total_k (patches), cols = prototypes ---
    # n_cols = len(active_protos)
    # fig, axs = plt.subplots(total_k, n_cols,
    #                         figsize=(n_cols * 2, total_k * 2))
    # if total_k == 1: axs = np.expand_dims(axs, 0)
    # if n_cols == 1:  axs = np.expand_dims(axs, 1)

    # for col, proto in enumerate(active_protos):
    #     indices = proto_to_indices[proto]
    #     for row in range(total_k):
    #         ax = axs[row, col]
    #         if row >= len(indices):
    #             ax.axis('off')
    #             continue

            patch_idx = indices[row]
            x, y = coords[patch_idx]
            pattern = f'*_x{int(x)}_y{int(y)}.npy'
            matches = glob.glob(os.path.join(patch_dir, pattern))
            if not matches:
                ax.axis('off'); continue

            patch = np.load(matches[0]).astype(np.uint8)
            ax.imshow(patch)
            ax.set_xticks([]); ax.set_yticks([]); ax.axis('off')

            # column titles on top row
            if row == 0:
                ax.set_title(f'Proto {proto}', fontsize=12, pad=4)

            # put a "Random" row label in the first column, but as free text
            if row == top_k:
                any_col = 0  # works even if this col is empty; position still exists
                row_pos = axs[top_k, any_col].get_position()
                y_center = (row_pos.y0 + row_pos.y1) / 2.0

                # left margin: match your subplots_adjust(left=...)
                fig.text(0.015, y_center, 'Random ↓', ha='left', va='center',
                        fontsize=12, fontweight='bold')
                # # text in axis coordinates, slightly outside on the left
                # ax.text(-0.15, 0.5, 'Random ↓',
                #         transform=ax.transAxes, va='center', ha='right',
                #         fontsize=12, fontweight='bold')

    # keep margins so the left-side "Random" text isn't clipped
    plt.subplots_adjust(left=0.06, right=0.995, top=0.95, bottom=0.03,
                        wspace=0.02, hspace=0.04)

    if save_path:
        plt.savefig(save_path, dpi=300)
        print(f"✅ Saved patch visualization to: {save_path}")
    plt.close(fig)


In [19]:
import os
import pandas as pd
import torch

def run_all_patch_visualizations(
    feats_h5_dir,
    patch_root,
    csv_path,
    save_root,
    panther_encoder,
    top_k=4,
    patch_size=224
):
    """
    Automates prototype patch visualization across slides using Panther qq assignments.

    Args:
        feats_h5_dir (str): Path to the directory containing .h5 feature files.
        patch_root (str): Root path to the patch directory (organized by magnification/class).
        csv_path (str): Path to CSV file with columns ['Filename', 'Class', 'Magnification'].
        save_root (str): Directory where visualization images will be saved.
        panther_encoder (torch.nn.Module): Trained Panther encoder model.
        top_k (int): Number of top patches to visualize per prototype.
        patch_size (int): Size of each patch (default: 224).
    """
    os.makedirs(save_root, exist_ok=True)
    df = pd.read_csv(csv_path)
    df['Filename'] = df['Filename'].str.strip()

    for h5_file in os.listdir(feats_h5_dir):
        if not h5_file.endswith('.h5'):
            continue

        slide_id = h5_file.replace('.h5', '').strip()

        row = df[df['Filename'] == slide_id]
        if row.empty:
            print(f"⚠️ Slide {slide_id} not found in metadata CSV.")
            continue

        class_label = row['Class'].values[0]
        magnification = row['Magnification'].values[0]

        h5_feats_fpath = os.path.join(feats_h5_dir, h5_file)
        patch_dir = os.path.join(patch_root, f"{magnification}x", class_label, slide_id)

        if not os.path.isdir(patch_dir):
            print(f"❌ Patch directory not found: {patch_dir}")
            continue

        print(f"✅ Processing: {slide_id} | Class: {class_label} | Mag: {magnification}x")

        # Optional: organize outputs per slide
        slide_dir = os.path.join(save_root, slide_id.replace(' ', '_'))
        os.makedirs(slide_dir, exist_ok=True)
        save_path = os.path.join(slide_dir, '8_patches.png')

        try:
            visualize_top_k_patches_per_prototype_from_qq(
                h5_feats_fpath=h5_feats_fpath,
                patch_dir=patch_dir,
                panther_encoder=panther_encoder,
                top_k=top_k,
                patch_size=patch_size,
                save_path=save_path
            )

        except Exception as e:
            print(f"❌ Error processing {slide_id}: {e}")


In [29]:
import os
import pandas as pd
import torch

def run_all_patch_visualizations(
    feats_h5_dir,
    patch_root,
    csv_path,  # should be your test.csv with columns: slide_id,label
    save_root,
    panther_encoder,
    top_k=4,
    patch_size=224
):
    """
    Automates prototype patch visualization across slides using Panther qq assignments.
    Only processes slides listed in test CSV.
    """
    os.makedirs(save_root, exist_ok=True)

    # Load test slides from CSV
    df_test = pd.read_csv(csv_path)
    df_test['slide_id'] = df_test['slide_id'].str.strip()
    test_slides = set(df_test['slide_id'])

    for h5_file in os.listdir(feats_h5_dir):
        if not h5_file.endswith('.h5'):
            continue

        slide_id = h5_file.replace('.h5', '').strip()

        # Skip if not in test set
        if slide_id not in test_slides:
            continue

        # Infer class & mag from directory structure if needed
        # If you don't have this in test.csv, adjust as necessary
        # For now, we'll skip that metadata check and just assume patch dir exists
        patch_dir = None
        for root, dirs, files in os.walk(patch_root):
            if os.path.basename(root) == slide_id:
                patch_dir = root
                break
        if patch_dir is None:
            print(f"❌ Patch directory not found for: {slide_id}")
            continue

        print(f"✅ Processing: {slide_id}")

        # Optional: organize outputs per slide
        slide_dir = os.path.join(save_root, slide_id)
        os.makedirs(slide_dir, exist_ok=True)
        save_path = os.path.join(slide_dir, '8_patches.png')

        try:
            visualize_top_k_patches_per_prototype_from_qq(
                h5_feats_fpath=os.path.join(feats_h5_dir, h5_file),
                patch_dir=patch_dir,
                panther_encoder=panther_encoder,
                top_k=top_k,
                patch_size=patch_size,
                save_path=save_path
            )

        except Exception as e:
            print(f"❌ Error processing {slide_id}: {e}")


In [30]:
run_all_patch_visualizations(
    # feats_h5_dir=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide',
    feats_h5_dir=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\uniextracted_mag5x_patch224_fp\feats_h5',
    patch_root=r'C:\Users\Vivian\Documents\CONCH\all_patches\patches_5x',
    # csv_path=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\visualization\slides_list.csv',
    csv_path=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_k=0\test.csv',
    save_root=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\test_visualizations',
    panther_encoder=panther_encoder,
    top_k=4
)


✅ Processing: FA 56B

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.2569
✅ Saved patch visualization to: C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\test_visualizations\FA 56B\8_patches.png
✅ Processing: FA 57B

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 1.9535
✅ Saved patch visualization to: C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\test_visualizations\FA 57B\8_patches.png
✅ Processing: FA 58B

=== map_em called ===

=== mog_eval called ===

Prototype norms: 28.685 ± 1.907
Covariance range: 0.100 to 0.100
[PANTHER] Mean soft assignment entropy: 2.4151
✅ Saved patch visualization to: C:\Users\Vivian\Documents\PANTHER\PANTHER\features\test_slide\test_visualizations\FA 58B\8_patches.png
✅ Processing: FA 59B

=== map_em called ===

=== 

inspecting qq scores

In [30]:
import os
import h5py
import numpy as np
import torch

def inspect_qq_scores(slide_id, h5_dir, panther_encoder, top_k=5):
    """
    Prints the prototype soft assignment (qq) scores for patches in a given slide.

    Args:
        slide_id (str): Slide ID (e.g. 'PT 41 B').
        h5_dir (str): Directory containing .h5 feature files.
        panther_encoder (nn.Module): Loaded Panther encoder model.
        top_k (int): Number of top patches (by max qq) to display.
    """
    # Build H5 path
    h5_path = os.path.join(h5_dir, f"{slide_id}.h5")
    if not os.path.isfile(h5_path):
        print(f"❌ H5 file not found: {h5_path}")
        return

    # Load features and coords
    with h5py.File(h5_path, 'r') as h5:
        feats = torch.tensor(h5['features'][:])  # [N, D]
        coords = h5['coords'][:]                 # [N, 2]

    print(f"✅ Loaded {feats.shape[0]} patches from {slide_id}")

    # Run Panther encoder to get qq
    with torch.inference_mode():
        info = panther_encoder.representation(feats.unsqueeze(0))
        qq = info['qq'][0, :, :, 0].cpu().numpy()  # shape: [N_patches, n_prototypes]

    print(f"qq shape: {qq.shape}")
    print(f"Prototype softmax scores for first {top_k} patches:")

    # Check top_k patches with their max score and full row
    for i in range(min(top_k, len(qq))):
        patch_scores = qq[i]
        max_score = patch_scores.max()
        proto = patch_scores.argmax()
        coord = coords[i]
        print(f"Patch {i} @ coord {tuple(coord)} → max proto: {proto}, max score: {max_score:.4f}, full: {np.round(patch_scores, 3)}")

    # Check if any patch has all 1.0
    suspicious = np.where(np.all(qq == 1.0, axis=1))[0]
    if len(suspicious) > 0:
        print(f"⚠️ {len(suspicious)} patches have qq scores all equal to 1.0 (possible bug).")



In [ ]:
# Example usage
slide_id = 'PT 41 B'
h5_dir = r'C:\Users\Vivian\Documents\CLAM\CLAM\FEATURES_DIR_5x\FEATURES_DIR_2.5x\uniextracted_mag2x_patch224_fp\feats_h5'
inspect_qq_scores(slide_id, h5_dir, panther_encoder)

temp fix to normalize prototypes

In [2]:
import numpy as np
import pickle
import tempfile

# Original prototypes
proto_path = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\splits\FA_PT_k=0\prototypes\prototypes_c16_uni_kmeans_num_1.0e+06.pkl'

# Load and normalize
with open(proto_path, 'rb') as f:
    proto_data = pickle.load(f)
    
prototypes = proto_data['prototypes']
prototypes = prototypes / np.linalg.norm(prototypes, axis=-1, keepdims=True)
prototypes = prototypes * 1.0  # Match UNI scale

# Create temporary modified prototype file
temp_proto_path = tempfile.NamedTemporaryFile(suffix='.pkl', delete=False).name
with open(temp_proto_path, 'wb') as f:
    pickle.dump({'prototypes': prototypes}, f)

In [3]:
### Loading PANTHER Encoder
panther_encoder = get_panther_encoder(in_dim=1024, p=16, proto_path=temp_proto_path, config_dir=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs', model_config='PANTHER_fa_pt', out_type='allcat')

📌 Loaded config path: C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs\PANTHER_fa_pt\config.json
📌 Loaded config_dict from JSON: {'in_dim': 1024, 'n_classes': 2, 'heads': 1, 'em_iter': 1, 'tau': 0.001, 'ot_eps': 0.1, 'n_fc_layers': 0, 'dropout': 0.25, 'out_type': 'allcat', 'out_size': 8, 'load_proto': False, 'proto_path': '.', 'fix_proto': False}


In [4]:
def get_modified_panther_encoder():
    # Get original encoder
    encoder = get_panther_encoder(
        in_dim=1024,
        p=16,
        proto_path=temp_proto_path,
        config_dir=r'C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs',
        model_config='PANTHER_fa_pt', 
        out_type='allcat'
    )
    
    # Modify covariance scaling directly
    with torch.no_grad():
        encoder.panther.priors.V_.data.fill_(np.log(np.exp(10.0) - 1))  # Sets base scale to ~10.0
    
    return encoder

panther_encoder = get_modified_panther_encoder()

📌 Loaded config path: C:\Users\Vivian\Documents\PANTHER\PANTHER\src\configs\PANTHER_fa_pt\config.json
📌 Loaded config_dict from JSON: {'in_dim': 1024, 'n_classes': 2, 'heads': 1, 'em_iter': 1, 'tau': 0.001, 'ot_eps': 0.1, 'n_fc_layers': 0, 'dropout': 0.25, 'out_type': 'allcat', 'out_size': 8, 'load_proto': False, 'proto_path': '.', 'fix_proto': False}


In [5]:
import torch.nn.functional as F

# Check prototypes
print("Prototype norms:", torch.norm(panther_encoder.panther.priors.m, dim=1))

# Check covariance
V = 0.1 * F.softplus(panther_encoder.panther.priors.V_)
print("Covariance range:", V.min().item(), "to", V.max().item())

Prototype norms: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       dtype=torch.float64)
Covariance range: 1.0 to 1.0


og proto map code. check why its only printing out 1

In [7]:
prototypes = panther_encoder.panther.priors.m.detach().cpu()
print("Prototype norms:", torch.norm(prototypes, dim=1))
# Should be in similar range as your normalized features

Prototype norms: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       dtype=torch.float64)


In [6]:
from mil_models.tokenizer import PrototypeTokenizer
import os
import numpy as np

h5_dir = r'C:\Users\Vivian\Documents\PANTHER\PANTHER\features\uniextracted_mag5x_patch224_fp\feats_h5'
h5_feats_path = os.path.join(h5_dir, 'FA 47 B1.h5')
# 1. Load patch features
with h5py.File(h5_feats_path, 'r') as h5:
    coords = h5['coords'][:]
    feats = torch.Tensor(h5['features'][:])  # shape: [N, D]

    feats = feats - feats.mean(dim=0)  # Center features
    feats = feats / (feats.std(dim=0) + 1e-6)  # Scale features
    print("Normalized feature stats - Mean:", feats.mean().item(), "Std:", feats.std().item())
    # print shape of feats and coords
    print('feats.shape', feats.shape)  # debug

# og code
with torch.inference_mode():
    info = panther_encoder.representation(feats.unsqueeze(dim=0))
    qqs = info['qq']
    # ---------
    # After getting qq, before argmax:
    print("Assignment scores before argmax (first 5 patches):")
    print(qqs[:5])  # Should show soft values if working properly

    # Check if values are actually one-hot or just very close
    # print("Unique values in qqs:", np.unique(qqs.round(decimals=3)))
   
    print('qqs.shape before tokenizer', qqs.shape) # debug
    # print('qqs before tokenizer', qqs)
    out = info['repr']
    print('out.shape', out.shape) # debug
    tokenizer = PrototypeTokenizer(p=16, out_type='allcat')
    mus, pis, sigmas = tokenizer.forward(out)
    print('mus.shape', mus.shape)  # debug
    print(mus)
    print('pis.shape', pis.shape)  # debug
    print(pis)
    print('sigmas.shape', sigmas.shape)  # debug
    mus = mus[0].detach().cpu().numpy()
    print('mus.shape after mus', mus.shape)  # debug
    print(mus[:5])  # debug
    qq = qqs[0,:,:,0].cpu().numpy()
    print('qq.shape after qq', qq.shape)  # debug
    global_cluster_labels = qq.argmax(axis=1)
    print('global_cluster_labels.shape', global_cluster_labels.shape)  # debug
    print('cluster_labels', global_cluster_labels[:5])  # debug
print(len(qq), "patches with soft assignments")
# Check how many prototypes get used in your data
unique_assignment_counts = np.bincount(global_cluster_labels)
print("Prototype usage:", unique_assignment_counts)

# If >90% patches assign to <3 prototypes, needs modification

for i in range(min(5, len(qq))):
    coord = coords[i]
    scores = qq[i]
    max_idx = scores.argmax()
    print(f"Patch {i} @ coord {tuple(coord)} → max proto: {max_idx}, score: {scores[max_idx]:.4f}, full: {np.round(scores, 3)}")

# Ensure pis is [N, p] numpy array
if isinstance(pis, torch.Tensor):
    pis_np = pis.detach().cpu().numpy()
    if pis_np.ndim == 3:
        pis_np = pis_np[0]
else:
    pis_np = pis

for i in range(min(5, len(qq))):
    coord = coords[i]
    scores = pis_np[i]
    max_idx = scores.argmax()
    print(f"Patch {i} @ coord {tuple(coord)} → max proto: {max_idx}, score: {scores[max_idx]:.4f}, full: {np.round(scores, 3)}")

for i in range(min(5, len(pis_np))):
    coord = coords[i]
    scores = pis_np[i]
    max_idx = scores.argmax()
    print(f"Patch {i} @ coord {tuple(coord)} → max proto: {max_idx}, score: {scores[max_idx]:.4f}, full: {np.round(scores, 3)}")


Normalized feature stats - Mean: 1.3673350229836956e-09 Std: 0.9997648000717163
feats.shape torch.Size([2136, 1024])

=== map_em called ===

=== mog_eval called ===

Prototype norms: 1.000 ± 0.000
Covariance range: 1.000 to 1.000
Assignment scores before argmax (first 5 patches):
tensor([[[[3.6000e-03],
          [3.7793e-06],
          [1.3385e-06],
          ...,
          [5.9588e-06],
          [2.5002e-01],
          [6.9040e-01]],

         [[6.1873e-05],
          [1.8758e-08],
          [1.0963e-07],
          ...,
          [1.3474e-07],
          [9.0918e-01],
          [8.6456e-02]],

         [[5.3165e-03],
          [3.0672e-07],
          [6.5236e-07],
          ...,
          [1.0985e-06],
          [1.5935e-04],
          [1.2452e-01]],

         ...,

         [[1.2213e-05],
          [2.3733e-03],
          [3.6421e-06],
          ...,
          [9.1788e-01],
          [1.0059e-04],
          [2.4233e-05]],

         [[3.3387e-04],
          [4.8458e-05],
          [6